# Задание 1

(**NB.** для запуска примеров кода нужен Python версии не ниже **3.10**, допускается использование других версий, в этом случае нужно самостоятельно избавиться от конструкции `match`).

Есть следующий код для [автоматического дифференцирования](https://en.wikipedia.org/wiki/Automatic_differentiation), в котором используются особенности системы типов языка `Python`: 

In [1]:
from dataclasses import dataclass
from typing import Union, Callable
from numbers import Number

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
         match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)    

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
 

def diff(func: Callable[[float], float]) -> Callable[[float], float]:
    return lambda x: func(Dual(x, 1.0)).d 

Поддерживаются две операции - сложение и умножение. Применить можно так:

In [2]:
# Функция, которую будем дифференцировать
def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

f_diff = diff(f)

# значение производной в точке x = 2
f_diff(2)

22.0

## Задание 1.1 (5 баллов)

Какие недостатки вы видите в данной реализации? Реализуйте поддержку (полностью самостоятельно или модифицируя приведенный код):
- [унарных операций](https://docs.python.org/3/reference/datamodel.html#object.__neg__) 
- деления
- возведения в степень

Каким образом можно проверить корректность решения?  Реализуйте достаточный, по вашему мнению, набор тестов.

In [3]:
# недостатки - скорость работы, ограниченность мат аппаратом и самой функцией
import math

@dataclass
class Dual:
    value: float
    d: float

    def __add__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value + o_value, self.d + o_d)
            case Number():
                return Dual(float(other) + self.value, self.d)

    def __mul__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value * o_value, self.value * o_d + self.d * o_value)
            case Number():
                return Dual(float(other) * self.value, float(other) * self.d)

    def __neg__(self) -> "Dual":
        return Dual(-self.value, -self.d)
    def __pos__(self) -> "Dual":
        return self

    def __truediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(self.value / o_value, (self.d * o_value - self.value * o_d) / (o_value * o_value))
            case Number():
                return Dual(self.value / float(other),  self.d / float(other))
    def __rtruediv__(self, other: Union["Dual", Number]) -> "Dual":
        match other:
            case Dual(o_value, o_d):
                return Dual(o_value / self.value, (self.value * o_d - self.d * o_value) / (self.value * self.value))
            case Number():
                return Dual(float(other) / self.value,  -float(other) / (self.value * self.value) * self.d)

    def __pow__(self, other: Union["Dual", Number], modulo=None):
        match other:
            case Dual(o_value, o_d):
                assert False, "not supported"
            case Number():
                return Dual(self.value ** float(other), float(other) * self.value ** (float(other) - 1))
    def __rpow__(self, other: Union["Dual", Number], modulo=None):
        match other:
            case Dual(o_value, o_d):
                assert False, "not supported"
            case Number():
                return Dual(float(other) ** self.value, math.log(float(other), math.e) *  float(other) ** self.value)

    __rmul__ = __mul__  # https://docs.python.org/3/reference/datamodel.html#object.__mul__
    __radd__ = __add__  # https://docs.python.org/3/reference/datamodel.html#object.__radd__
    
    
assert diff(lambda x: 5 * x * x + 2 * x + 2)(2) == 22
assert diff(lambda x: -x + 2 * +x + 2)(2) == 1
assert diff(lambda x: 5 / x  +  x / 2 + x / x + 2)(2) == -0.75
assert (diff(lambda x: x ** 3 + 3 ** x + 2)(2) - 21.887511) < 0.00001

## Задание 1.2 (7 баллов)
Придумайте способ и реализуйте поддержку функций:
- `exp()`
- `cos()`
- `sin()`
- `log()`

Добавьте соответствующие тесты

In [4]:
class dmath:
    @staticmethod
    def exp(x: Union["Dual", Number]) -> Union["Dual", Number]:
        match x:
            case Dual(value, d):
                return Dual(math.exp(value), d * math.exp(value))
            case Number():
                return math.exp(x)

    @staticmethod
    def cos(x: Union["Dual", Number]) -> Union["Dual", Number]:
        match x:
            case Dual(value, d):
                return Dual(math.cos(value), -math.sin(value) * d)
            case Number():
                return math.cos(x)

    @staticmethod
    def sin(x: Union["Dual", Number]) -> Union["Dual", Number]:
        match x:
            case Dual(value, d):
                return Dual(math.sin(value), math.cos(value) * d)
            case Number():
                return math.sin(x)

    @staticmethod
    def log(x: Union["Dual", Number]) -> Union["Dual", Number]:
        match x:
            case Dual(value, d):
                return Dual(math.log(value), d / value)
            case Number():
                return math.log(x)
            
assert (diff(lambda x: dmath.exp(x) * x)(2) - 22.167168) < 0.00001
assert(diff(lambda x: dmath.cos(x) * x)(2) - -2.234742) < 0.00001
assert (diff(lambda x: dmath.sin(x) * x)(2) - 0.077004) < 0.00001
assert (diff(lambda x: dmath.log(x) * x)(2) - 1.693147) < 0.00001
assert (diff(lambda x: dmath.exp(dmath.sin(dmath.cos(dmath.log(x)))) * x)(2) - 1.084534) < 0.00001

## Задание 1.3 (3 балла)

Воспользуйтесь методами **численного** дифференцирования для "проверки" работы кода на нескольких примерах. Например,  библиотеке `scipy` есть функция `derivative`. Или реализуйте какой-нибудь метод численного дифференцирования самостоятельно (**+5 баллов**)

In [5]:
from scipy.misc import derivative

def f(x: float) -> float:
    return 5 * x * x + 2 * x + 2

derivative(f, 2.)

22.0

In [6]:
def numerical_derivative_1d(func):
    def deriv_func(x):
        dx=0.0000001
        dy=(func(x+dx)-func(x-dx)) / (2*dx)
        return dy
        
    return deriv_func

numerical_derivative_1d(f)(2.)

21.99999997287705

## Задание 1.4 (10 баллов)

Необходимо разработать систему автоматического тестирования алгоритма дифференцирования в следующем виде:
- реализовать механизм генерации "случайных функций" (например, что-то вроде такого: $f(x) = x + 5 * x - \cos(20 * \log(12 - 20 * x * x )) - 20 * x$ )
- сгенерировать достаточно большое число функций и сравнить результаты символьного и численного дифференцирования в случайных точках 

Генерацию случайных функций можно осуществить, например, двумя путями. 
1. Генерировать функцию в текстовом виде, зачем использовать встроенную функцию [eval](https://docs.python.org/3/library/functions.html#eval)

```python
func = eval("lambda x: 2 * x + 5")
assert func(42) == 89 
```

2. Использовать стандартный модуль [ast](https://docs.python.org/3/library/ast.html), который позволяет во время выполнения программы манипулировать [Абстрактным Синтаксическим Деревом](https://ru.wikipedia.org/wiki/%D0%90%D0%B1%D1%81%D1%82%D1%80%D0%B0%D0%BA%D1%82%D0%BD%D0%BE%D0%B5_%D1%81%D0%B8%D0%BD%D1%82%D0%B0%D0%BA%D1%81%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE).
Например, выражение 

```python
func = lambda x: 2 * x + 5
```

Можно запрограммировать с помощью кода:

```python

expr = ast.Expression(
    body=ast.Lambda(
        args=ast.arguments(
            args=[
                ast.arg(arg='x')
            ],
            posonlyargs=[],
            kwonlyargs=[],
            kw_defaults=[],
            defaults=[]
        ),
        body=ast.BinOp(
            left=ast.BinOp(
                left=ast.Constant(value=2),
                op=ast.Mult(),
                right=ast.Name(id='x', ctx=ast.Load())
            ),
            op=ast.Add(),
            right=ast.Constant(value=5)
        )
    )
)

ast.fix_missing_locations(expr)

func = eval(compile(expr, filename="", mode="eval"))

assert func(42) == 89
```

При реализации нужно учитывать области допустимых значений функций.

In [7]:
import random

def rand_function():
    s = "lambda x: "
    signs = [' + ', ' * ', ' / ']
    brackets = '()'
    fs = ['dmath.sin(', 'dmath.cos(', 'dmath.log(', 'dmath.exp(']
    fs2 = ['dmath.sin(', 'dmath.cos(']
    pow_sign = ' ** '

    for i in range(random.randint(1, 100)):
        op = random.randint(0, 4)
        if op == 0:
            if random.randint(0, 1) == 0:
                s += 'x' + random.choice(signs)
            else:
                s += random.choice(fs2)
                if random.randint(0, 1) == 0:
                    s += 'x' + brackets[1] + random.choice(signs)
                else:
                    s += random.choice(fs2) + 'x' + brackets[1] + brackets[1] + random.choice(signs)
        elif op == 1:
            s += str(random.uniform(-10, 10)) + random.choice(signs)
        elif op == 2:
            s += random.choice(fs) + 'x' + brackets[1]  + random.choice(signs)
        elif op == 3:
            s += 'x' + pow_sign + str(random.uniform(-3, 3)) + random.choice(signs)
        elif op == 4:
            s += str(random.uniform(-3, 3)) + pow_sign + 'x' + random.choice(signs)


    s += 'x'
    return s

for i in range(5000):
    r = rand_function()
    point = random.uniform(0.1, 10)
    ll = diff(eval(r))(point)
    rr = numerical_derivative_1d(eval(r))(point)
    # немного сложное условие, чтобы не было проблем с пограничными случаями для очень больших и очень маленьких чисел
    assert abs(ll - rr) <= 0.001 * max(abs(ll), abs(rr)) or abs(ll - rr) <= 0.001, str(ll) + " " + str(rr)

print("all tested ok!")

all tested ok!


## Задание 1.5 (7 баллов)

Реализуйте поддержку функций нескольких аргументов. Например

```python
def f(x: float, y: float, z: float) -> float:
    return x * y + z - 5 * y  


f_diff = diff(f)

f_diff(10, 10, 10) # = [10, 5, 1]
```

In [8]:
def diff_multy(func: Callable):
    def r(*args, **kwargs):
        if kwargs:
            ans = []
            for i, v in kwargs.items():
                wer = set(kwargs.items())
                erw = set({i: v}.items())
                diff = dict(wer - erw)

                diff[i] = Dual(v, 1.0)

                p = func(**diff).d
                ans.append(p)
            return ans
        else:
            assert args
            ans = []
            for i in range(len(args)):
                diff = list(args)
                diff[i] = Dual(args[i], 1.0)
                diff = tuple(diff)

                p = func(*diff).d
                ans.append(p)
            return ans
    return r


In [9]:
def f_multy(x,y,z):
    return x*x + y**3 + z**4

f_multy_d = diff_multy(f_multy)
print(f_multy_d(1, 1, 1))
print(f_multy_d(x=1, y=1, z=1))

[2.0, 3.0, 4.0]
[2.0, 3.0, 4.0]
